### Users Table
- `id`: Unique identifier for each user (numeric)
- `created_at`: User creation timestamp (ISO date string)
- `attribution_source`: User acquisition source (tiktok, instagram, or organic)
- `country`: User's country (US, TR, or NL)
- `name`: User's name

### User Events Table
- `id`: Unique event identifier (numeric)
- `created_at`: Event timestamp (ISO date string)
- `user_id`: Reference to users table (numeric)
- `event_name`: Type of event (app_install, trial_started, trial_cancelled, subscription_started, subscription_renewed, subscription_cancelled)
- `amount_usd`: Transaction amount in USD (numeric)

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [5]:
try:
    conn = sqlite3.connect('papcorns.sqlite')
except Exception as e:
    print(e)

In [7]:
users_df = pd.read_sql_query("SELECT * FROM users LIMIT 5;", conn)
print("Users table preview:")
display(users_df)

# Load events table
events_df = pd.read_sql_query("SELECT * FROM user_events LIMIT 5;", conn)
print("\nUser events table preview:")
display(events_df)

Users table preview:


,id,created_at,attribution_source,country,name
0,1,2024-05-07T00:00:00,instagram,US,Eve Brown
1,2,2024-10-12T00:00:00,instagram,NL,Frank Moore
2,3,2024-10-15T00:00:00,tiktok,TR,Ivy Anderson
3,4,2024-08-28T00:00:00,tiktok,TR,Alice Brown
4,5,2024-04-03T00:00:00,organic,NL,Bob Moore



User events table preview:


,id,created_at,user_id,event_name,amount_usd
0,1,2024-05-07T00:00:00,1,app_install,None
1,2,2024-05-12T00:00:00,1,trial_started,None
2,3,2024-05-24T00:00:00,1,trial_cancelled,None
3,4,2024-10-12T00:00:00,2,app_install,None
4,5,2024-10-13T00:00:00,2,trial_started,None


In [25]:
users_df = pd.read_sql_query("SELECT * FROM users", conn)

In [26]:
events_df = pd.read_sql_query("SELECT * From user_events ", conn)

In [29]:
users_df.describe

<bound method NDFrame.describe of         id           created_at attribution_source country           name
0        1  2024-05-07T00:00:00          instagram      US      Eve Brown
1        2  2024-10-12T00:00:00          instagram      NL    Frank Moore
2        3  2024-10-15T00:00:00             tiktok      TR   Ivy Anderson
3        4  2024-08-28T00:00:00             tiktok      TR    Alice Brown
4        5  2024-04-03T00:00:00            organic      NL      Bob Moore
...    ...                  ...                ...     ...            ...
997    998  2025-02-01T00:00:00          instagram      TR      Bob Davis
998    999  2024-12-24T00:00:00            organic      NL  Charlie Davis
999   1000  2025-02-13T00:00:00            organic      NL  Jack Anderson
1000  1001  2025-02-16T00:00:00          instagram      US    Bruce Wayne
1001  1002  2025-02-16T00:00:00            organic      TR     Clark Kent

[1002 rows x 5 columns]>

In [30]:
events_df.describe

<bound method NDFrame.describe of         id           created_at  user_id            event_name  amount_usd
0        1  2024-05-07T00:00:00        1           app_install         NaN
1        2  2024-05-12T00:00:00        1         trial_started         NaN
2        3  2024-05-24T00:00:00        1       trial_cancelled         NaN
3        4  2024-10-12T00:00:00        2           app_install         NaN
4        5  2024-10-13T00:00:00        2         trial_started         NaN
...    ...                  ...      ...                   ...         ...
3481  3482  2025-02-25T00:00:00     1000       trial_cancelled         NaN
3482  3483  2025-02-25T00:00:00     1001           app_install         NaN
3483  3484  2025-02-25T00:00:00     1001         trial_started         NaN
3484  3485  2025-02-25T00:00:00     1001  subscription_started        9.99
3485  3486  2025-02-25T00:00:00     1002           app_install         NaN

[3486 rows x 5 columns]>

Let's start with checking missing values

In [31]:
users_df.isna().any()

id                    False
created_at            False
attribution_source    False
country               False
name                  False
dtype: bool

In [32]:
events_df.isna().any()

id            False
created_at    False
user_id       False
event_name    False
amount_usd     True
dtype: bool

There are missing values 

In [33]:
events_df.isna().sum()

id               0
created_at       0
user_id          0
event_name       0
amount_usd    2255
dtype: int64

It turns out, 2255 rows of the 3486 are missing in the `amount_usd` column